In [ ]:
import pandas as pd
import seaborn as sns

df1 = pd.read_csv(r'/kaggle/input/patient-data/synthetic_facility_v3.csv')

In [ ]:
print("Sample of the facility dataset")
df1.head()

## Data cleaning

In [ ]:
#Cleaning the facility dataset
#edit the formulation of dob
df1['dob'] = pd.to_datetime(df1['dob'],errors = 'coerce')

#change the format of date in column dob to 9/9/1930
df1['dob'] = df1['dob'].dt.strftime('%d/%m/%Y')
print(df1)

import warnings
warnings.filterwarnings('ignore', category=Warning)

In [ ]:
#formating the visit date
df1['visitdate'] = pd.to_datetime(df1['visitdate'],errors = 'coerce')

#organizing the format of visittime column
df1['visitdate'] = df1['visitdate'].dt.strftime('%d/%m/%Y')
df1

In [ ]:
#check columns with null values
null_columns = df1.isnull().any()

#Display coulmns with null values
columns_with_nulls = null_columns[null_columns]
print("Columns with null values")
print(columns_with_nulls)

In [ ]:
#check for leading spaces in the first name column
leading_spaces = df1['firstname'].str.startswith(' ')

#remove any leading spaces
df1['firstname']=df1['firstname'].str.strip()

#output leading spaces information
print("Leading spaces:")
print(leading_spaces)

In [ ]:
#check for leading spaces in the last name column
leading_spaces1 = df1['lastname'].str.startswith(' ')

#remove any leading spaces
df1['lastname']=df1['lastname'].str.strip()

#output leading spaces information
print("Leading spaces:")
print(leading_spaces1)

In [ ]:
#Cleaning the hdss dataset
df2 = pd.read_csv(r'/kaggle/input/patient-data/synthetic_hdss_v3.csv')
df2

In [ ]:
#edit the dob format in the hdss dataset

df2['dob'] = pd.to_datetime(df2['dob'],errors = 'coerce')

#change the format of date in column dob to 9/9/1930
df2['dob'] = df2['dob'].dt.strftime('%d/%m/%Y')
df2

In [ ]:
#check for any leading spaces in first name column

leading_spaces2 = df2['firstname'].str.startswith(' ')

#remove any leading spaces
df2['firstname']=df2['firstname'].str.strip()

#output leading spaces information
print("Leading spaces:")
print(leading_spaces2)

In [ ]:
#check for any leading spaces in last name column

leading_spaces3 = df2['lastname'].str.startswith(' ')

#remove any leading spaces
df2['lastname']=df2['lastname'].str.strip()

#output leading spaces information
print("Leading spaces:")
print(leading_spaces3)

# indexing 
Determining the records where by they are the same entity ie, same last names
This gives us the same last names .csv files name
A different function is used to generate the "people in facility only" dataset

These files are used to test our implementation

**matching_last_names.csv**

**people_in_facility_only.csv**

In [ ]:
# Merge the two datasets on the common column
common_dataset = pd.merge(df1, df2, on='firstname', how='inner')

# Drop duplicates based on the common column
common_dataset.drop_duplicates(subset='firstname', inplace=True)

# Reset index
common_dataset.reset_index(drop=True, inplace=True)

# Display the combined dataset
common_dataset

In [ ]:
import pandas as pd

# Assuming 'common_dataset' is the merged DataFrame
# Check for rows where the last names match in both 'lastname_x' and 'lastname_y'
matching_last_names = common_dataset[common_dataset['lastname_x'] == common_dataset['lastname_y']]

# Drop rows where the last names differ
matching_last_names.dropna(subset=['lastname_x', 'lastname_y'], inplace=True)

# Display the DataFrame with matching last names
matching_last_names

# Fellegi Sunter

### Probabilistic machine learning approach

In [ ]:
import math
from metaphone import doublemetaphone  

def default_match_fn(a, b):
    """
    Default function for determining if two values are a match.
    """
    if isinstance(a, str) and isinstance(b, str):
        return a.lower() == b.lower()
    else:
        return a == b

def names_match(name_a, name_b):
    """
    Do name_a or name_b share any matching phonetic values via double-metaphone?
    """
    if pd.isna(name_a) or pd.isna(name_b):
        return False  # Handle NaN values, consider them as not matching
    else:
        metaphones_a = doublemetaphone(name_a)
        metaphones_b = doublemetaphone(name_b)
        return any(m_a == m_b for m_a in metaphones_a for m_b in metaphones_b)

identifier_fields = [
    {"key": "firstname", "match_prob": 0.90, "unmatch_prob": 0.10},
    {"key": "lastname", "match_prob": 0.95, "unmatch_prob": 0.01},
    {"key": "petname", "match_fn": names_match, "match_prob": 0.60, "unmatch_prob": 0.20},
    {"key": "dob", "match_fn": names_match, "match_prob": 0.90, "unmatch_prob": 0.10},
    {"key": "sex", "match_prob": 0.95, "unmatch_prob": 0.05}
]

def compare_patients(patient_a, patient_b, identifier_fields):
    """
    Compare two patients and return a match score using the Fellegi-Sunter method.
    """
    weight = 0
    
    for field_info in identifier_fields:
        key = field_info["key"]
        field_a_value = patient_a.get(key)
        field_b_value = patient_b.get(key)
        
        if field_a_value is None or field_b_value is None:
            continue
        
        match_fn = field_info.get("match_fn", default_match_fn)
        is_a_match = match_fn(field_a_value, field_b_value)
        
        match_prob = field_info["match_prob"]
        unmatch_prob = field_info["unmatch_prob"]
        
        if is_a_match:
            weight += math.log(match_prob / unmatch_prob)
        else:
            weight += math.log((1 - match_prob) / (1 - unmatch_prob))
    
    return weight

In [ ]:
# Iterate over each pair of rows in the two dataframes and compute match score
# runtime for this is approximately 1 hour
for indexA, rowA in df1.iterrows():
    for indexB, rowB in df2.iterrows():
        match_score = compare_patients(rowA, rowB, identifier_fields)
        #print(f"Match score between {rowA['firstname']} {rowA['lastname']} (index: {indexA}) and {rowB['firstname']} {rowB['lastname']} (index: {indexB}): {match_score}")

In [ ]:
#Establishing the highest score and lowest score so that we can classify and analyse the data
#runtime for this is also around 1 hour
lowest_score = float('inf')
highest_score = float('-inf')
lowest_score_row = None
highest_score_row = None

for indexA, rowA in df1.iterrows():
    for indexB, rowB in df2.iterrows():
        match_score = compare_patients(rowA, rowB, identifier_fields)
        if match_score < lowest_score:
            lowest_score = match_score
            lowest_score_row = (rowA, rowB)
        if match_score > highest_score:
            highest_score = match_score
            highest_score_row = (rowA, rowB)

print("Lowest match score:", lowest_score)
print("Corresponding rows:", lowest_score_row)

print("Highest match score:", highest_score)
print("Corresponding rows:", highest_score_row)

# Evaluation
#### Search function is used

**Patients in hospital data only**
1. shakalu
2. kalabako

**Patients in both datasets**
1. Dukain
2. Nasiri
3. Bakali

*I havent found patients whose data does not need to be edited, most dates are conflicting*

> The perfect matching score is 12.99137731410753 as per now, but with conflicting dates for most patients
> 
> The scores between the range 10 to 12 need to be further evaluated to figure out where the errors or the differences are ie phonetics or mispellings
> 
> A much better testing method needs to be reviewed

In [ ]:
# this is a search function
# enter a name randomly or choose from the above names and a corresponding message will appear
def search_patients4(match_score):
    highest_score = float('-inf')
    lowest_score = float('inf')
    matching_pair = None
    unmatching_pair = None
    similar_data = None
    search_name = input("Enter patient name: ").strip()
    if not search_name:
        print("Please enter a name to search.")
        return

    for indexA, rowA in df1.iterrows():
        if search_name.lower() in str(rowA['firstname']).lower():
            for indexB, rowB in df2.iterrows():
                current_score = compare_patients(rowA, rowB, identifier_fields)
                if current_score >=10:  # Filtering out patients with a score of 12 and above
                    if current_score > highest_score:
                        highest_score = current_score
                        similar_data = (rowA, rowB)
                elif current_score >= 12.99137731410753:  # Filtering out patients with a score of 12 and above
                    if current_score > highest_score:
                        highest_score = current_score
                        matching_pair = (rowA, rowB)
                else:
                    lowest_score = current_score
                    unmatching_pair = (rowA, rowB)

    if similar_data:
        print("Highest matching score:", highest_score)
        print("Unmatching data, please review patient")
        print("Corresponding row for search name in df1:", similar_data[0])
        print("Corresponding row for search name in df2:", similar_data[1])
    elif matching_pair:
        print("Highest matching score:", highest_score)
        print("Patient both in HDSS and hospital data")
        print("Corresponding row for search name in df1:", matching_pair)
        
    elif unmatching_pair:
        print("Lowest matching score:", lowest_score)
        print("Patient in Hospital data only")
        print("Corresponding rows:", unmatching_pair[0])
    else:
        print("No match found for", search_name)

# Example usage:
match_score = 0  # Assuming you have some way to define the matching score
search_patients4(match_score)

# Linking to the frontend 🥳

In [2]:
import pickle

# Consolidate your logic into a single cell
record_linkage_logic = import pandas as pd
import seaborn as sns

df1 = pd.read_csv(r'/kaggle/input/patient-data/synthetic_facility_v3.csv')

print("Sample of the facility dataset")
df1.head()
#Cleaning the facility dataset
#edit the formulation of dob
df1['dob'] = pd.to_datetime(df1['dob'],errors = 'coerce')

#change the format of date in column dob to 9/9/1930
df1['dob'] = df1['dob'].dt.strftime('%d/%m/%Y')
print(df1)

import warnings
warnings.filterwarnings('ignore', category=Warning)

#formating the visit date
df1['visitdate'] = pd.to_datetime(df1['visitdate'],errors = 'coerce')

#organizing the format of visittime column
df1['visitdate'] = df1['visitdate'].dt.strftime('%d/%m/%Y')
df1


# Save the logic
with open('record_linkage_logic.pkl', 'wb') as f:
    pickle.dump(record_linkage_logic, f)

SyntaxError: invalid syntax (2167459012.py, line 4)

In [ ]:
# this is a search function
# enter a name randomly or choose from the above names and a corresponding message will appear
def search_patients4(match_score):
    highest_score = float('-inf')
    lowest_score = float('inf')
    matching_pair = None
    unmatching_pair = None
    similar_data = None
    search_name = input("Enter patient name: ").strip()
    if not search_name:
        print("Please enter a name to search.")
        return

    for indexA, rowA in df1.iterrows():
        if search_name.lower() in str(rowA['firstname']).lower():
            for indexB, rowB in df2.iterrows():
                current_score = compare_patients(rowA, rowB, identifier_fields)
                if current_score >=10:  # Filtering out patients with a score of 12 and above
                    if current_score > highest_score:
                        highest_score = current_score
                        similar_data = (rowA, rowB)
                elif current_score >= 12.99137731410753:  # Filtering out patients with a score of 12 and above
                    if current_score > highest_score:
                        highest_score = current_score
                        matching_pair = (rowA, rowB)
                else:
                    lowest_score = current_score
                    unmatching_pair = (rowA, rowB)

    if similar_data:
        print("Highest matching score:", highest_score)
        print("Unmatching data, please review patient")
        print("Corresponding row for search name in df1:", similar_data[0])
        print("Corresponding row for search name in df2:", similar_data[1])
    elif matching_pair:
        print("Highest matching score:", highest_score)
        print("Patient both in HDSS and hospital data")
        print("Corresponding row for search name in df1:", matching_pair)
        
    elif unmatching_pair:
        print("Lowest matching score:", lowest_score)
        print("Patient in Hospital data only")
        print("Corresponding rows:", unmatching_pair[0])
    else:
        print("No match found for", search_name)

# Example usage:
match_score = 0  # Assuming you have some way to define the matching score
search_patients4(match_score)